In [ ]:
# Data 802: Analytical Tools and Foundations
# Homework assignment 1: Part 1
# Last updated: June 14, 2018 by Anna M. Kot

# You are also given hourly weather information for DC area for the 
# last five years. The first objective is to merge all of this information to 
# create one flat file. You are also required to create a separate holiday/ 
# adverse event flag variable to be used in predictive modeling

# Import dependencies
import pandas as pd
import numpy as np
import datetime
import glob
import os

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

# Set the working directory to the folder location containing 
# the raw Capital Bikeshare trip data and the weather data. 
# The folder location containing the raw data should contain 
# Capital Bikeshare trip data for only quarters one through
# four for the years 2016 and 2017 (e.g., 2016Q1-capitalbikeshare-tripdata.csv).
os.chdir("/Users/annakot/Desktop/M.S. Analytics/Data 802/01. HW1/Raw Data")

# Import the eight raw Capital Bikeshare data files into 
# a single data frame named CBSdf.
CBSdf = pd.DataFrame([])
 
for counter, file in enumerate(glob.glob("201*")):
    namedf = pd.read_csv(file, skiprows=0, usecols=range(9))
    CBSdf = CBSdf.append(namedf)

# Import the raw weather data into a single data frame named Wdf.
Wdf = pd.DataFrame([])
 
for counter, file in enumerate(glob.glob("W*")):
    namedf = pd.read_csv(file, skiprows=0, usecols=range(28))
    Wdf = Wdf.append(namedf)
    
# Create an array, CBSdfArray, to segment year, month, date, 
# and hour from CBSdf and append to CBSdfArray.
CBSdfArray = np.array(CBSdf["Start date"])
year=[]
month=[]
date=[]
hour=[]

for i in CBSdfArray:
    j = datetime.datetime.strptime(i, "%Y-%m-%d %H:%M:%S")
    year.append(j.year)
    month.append(str(j.month).zfill(2))
    date.append(str(j.day).zfill(2))
    hour.append(str(j.hour).zfill(2))
    
# Create a new Year, Month, Day, and Hour column, respectively, in 
# CBSdf and append segmented year, month, date, and hour from CBSdfArray.    
CBSdf["Year"] = year
CBSdf["Month"] = month
CBSdf["Day"] = date
CBSdf["Hour"] = hour

# Create a new column in CBSdf called Date_Key, concatenating 
# Year, Month, Day, and Hour columns from CBSdf.
CBSdf["Date_Key"] = CBSdf["Year"].astype(str)+"-"+CBSdf["Month"].astype(str)+"-"+CBSdf["Day"].astype(str)+"-"+CBSdf["Hour"].astype(str)

# Create an array, WdfArray, to segment year, month, date, 
# and hour from Wdf and append to WdfArray.
WdfArray = np.array(Wdf["dt_iso"])
year=[]
month=[]
date=[]
hour=[]

for i in WdfArray:
    j = datetime.datetime.strptime(i, "%Y-%m-%d %H:%M:%S +0000 UTC")
    year.append(j.year)
    month.append(str(j.month).zfill(2))
    date.append(str(j.day).zfill(2))
    hour.append(str(j.hour).zfill(2))
    
# Create a new Year, Month, Day, and Hour column, respectively, in 
# Wdf and append segmented year, month, date, and hour from WdfArray. 
Wdf["Year"] = year
Wdf["Month"] = month
Wdf["Day"] = date
Wdf["Hour"] = hour

# Create a new column in Wdf called Date_Key, concatenating 
# Year, Month, Day, and Hour columns from Wdf.
Wdf["Date_Key"] = Wdf["Year"].astype(str)+"-"+Wdf["Month"].astype(str)+"-"+Wdf["Day"].astype(str)+"-"+Wdf["Hour"].astype(str)

# Create a dataframe, CBSdf_Grouped, illustrating the number 
# of bikes rented per hour, per station.
CBSdf_Grouped = pd.DataFrame (
      CBSdf.groupby(["Date_Key","Start station number"], sort=True)
        .size()
        .unstack(fill_value=0)
        .reset_index()
        .rename_axis(None, axis=1)
      )

# Create a dataframe, CBSdf_Counts, illustrating the total 
# number of bikes rented per hour.
CBSdf_Counts = pd.DataFrame (
      CBSdf.groupby(["Date_Key"], sort=True)["Start station number"].count()
      )

# Merge CBSdf_Counts into CBSdf_Grouped and rename the 
# 'Start station number' column to 'Total'.
CBSdf_Grouped = pd.merge(left=CBSdf_Grouped, right=CBSdf_Counts, left_on=["Date_Key"], how='left', right_index=True)
CBSdf_Grouped = CBSdf_Grouped.rename(columns={"Start station number": "Total"})

# Merge Wdf into CBSdf_Grouped to creat a new dataframe, CBSWdf, reflecting one dataframe.
CBSWdf = pd.merge(left=CBSdf_Grouped, right=Wdf, on=["Date_Key"], how='left')

# Convert temperature data from Kelvin to Fahrenheit.
CBSWdf["Temp_F"] = (9/5)*(CBSWdf['temp'] - 273) + 32
CBSWdf["Temp_min_F"] = (9/5)*(CBSWdf['temp_min'] - 273) + 32
CBSWdf["Temp_max_F"] = (9/5)*(CBSWdf['temp_max'] - 273) + 32

# Convert wind speed data from meters/second to miles/hour.
CBSWdf["Wind_speed_miles"] = CBSWdf["wind_speed"] * 2.2369

# Determine which dates are considered Federal US Holidays and return 'True' or 'False'
# in a new column if or if not a Federal US Holiday, respectively.
dr = pd.to_datetime(CBSWdf["Date_Key"])
cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())

CBSWdf['Is_Holiday'] = pd.to_datetime(CBSWdf["Date_Key"]).dt.date.astype('datetime64[ns]').isin(holidays)

# Drop unncessary columns
CBSWdf.drop(['dt', 'dt_iso', 'city_id','city_name','lat','lon','temp','temp_min','temp_max','sea_level','grnd_level','wind_speed','rain_1h','rain_3h','rain_24h','rain_today','snow_1h','snow_3h','snow_24h','snow_today','weather_icon'], axis=1, inplace=True)

# Export the CBSWdf dataframe to a specific folder location 
# and save as CapitalBikeShareWeather.csv.
CBSWdf.to_csv('/Users/annakot/Desktop/M.S. Analytics/Data 802/01. HW1/Raw Data/CapitalBikeShareWeather.csv', index=False)